In [51]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tensorflow.keras.layers import Input, Dense, concatenate

# Make NumPy printouts easier to read.
np.set_printoptions(precision=5, suppress=True)

import tensorflow as tf
print(tf.__version__)

2.13.0


In [52]:
dataset = pd.read_csv('data_obrobiona/calosc.csv')
dataset

,dzien_tygodnia,czas,przeplyw
0,4,0.000099,252
1,4,0.000198,78
2,4,0.000297,253
3,4,0.000396,192
4,4,0.000495,8
...,...,...,...
369504,2,0.999808,213
369505,2,0.999856,253
369506,2,0.999904,186
369507,2,0.999952,8


In [53]:
train_dataset = dataset.sample(frac=0.8, random_state=0)
test_dataset = dataset.drop(train_dataset.index)
x_train=train_dataset.drop(columns="przeplyw");
x_test=test_dataset.drop(columns="przeplyw");
y_train=train_dataset.przeplyw;
y_test=test_dataset.przeplyw;
train_data=x_train.drop(columns="czas")
train_czas=x_train.drop(columns="dzien_tygodnia")

In [54]:
train_czas.shape

(295607, 1)

In [55]:
model = tf.keras.models.Sequential([
      tf.keras.layers.Dense(126, activation='relu', input_shape=(2,)),
      tf.keras.layers.Dense(64, activation='relu'),
      tf.keras.layers.Dense(32, activation='relu'),
      tf.keras.layers.Dense(1,activation='linear') 
])

In [56]:


model.compile(optimizer=tf.optimizers.Adam(learning_rate=0.01),
              loss="mse",
              metrics=['mae'])

In [57]:
model.fit(x_train, y_train, epochs=5, batch_size=32)

Epoch 1/5


9238/9238 [==============================] - 28s 3ms/step - loss: 5554.4585 - mae: 64.2980
Epoch 2/5
9238/9238 [==============================] - 53s 6ms/step - loss: 5477.9399 - mae: 64.0134
Epoch 3/5
9238/9238 [==============================] - 9s 924us/step - loss: 5462.7202 - mae: 63.9558
Epoch 4/5
9238/9238 [==============================] - 7s 753us/step - loss: 5459.8843 - mae: 63.9405
Epoch 5/5
9238/9238 [==============================] - 7s 746us/step - loss: 5459.3130 - mae: 63.9324


In [58]:
model.evaluate(x_test,  y_test, verbose=2)

2310/2310 - 1s - loss: 5499.1763 - mae: 64.2051 - 1s/epoch - 569us/step


[5499.17626953125, 64.2051010131836]

In [59]:
sample_time=2;      #[s]
symulation_time=60; #[min]

current_hours=12;   #[h]
current_minutes=12; #[min]
current_day=1;      #[label, pon-1, wt-2,...,nd-7]

In [66]:
sample_time_normalized=sample_time/(60*60*24);              #[min - normalized]
symulation_time_normlized=symulation_time/(24*60)           #[min - normalized]
current_time=(current_hours*60+current_minutes)/(24*60)     #[min - normalized]

if current_time+symulation_time_normlized<1:
    flow_time=np.arange(current_time,current_time+symulation_time_normlized,sample_time_normalized) #[min - normalized]
    flow_time=np.vstack((current_day*np.ones_like(flow_time, dtype=int),flow_time)).T               #[label, min]

flow_time

array([[1.     , 0.50833],
       [1.     , 0.50836],
       [1.     , 0.50838],
       ...,
       [1.     , 0.54993],
       [1.     , 0.54995],
       [1.     , 0.54998]])

In [67]:
flow_result = model.predict(flow_time)                   # [l]
water_usage_predict=np.sum(flow_result)*(sample_time/60) # [l]

water_usage_predict

57/57 [==============================] - 0s 699us/step


7451.5875